In [1]:
import time
start = time.time()

from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext.getOrCreate();

fileName = ('data.txt')
numPartitions = 4;
minSupport = 3;

dados = sc.textFile(fileName);
arquivoRDD = sc.parallelize(dados.collect(), numPartitions) \
               .map(lambda x: x.split('\t'))

print(arquivoRDD.take(10));
print(type(arquivoRDD));


[['1,101,102,103'], ['2,101,102,103,104,105'], ['3,101,103,104'], ['4,101,102,104,105'], ['5,102,103,104'], ['6,103,105,106,107,108'], ['7,103,105,106,107,110'], ['8,104,106,107,109,110'], ['9,106,108,110'], ['10,102,103,107,108,109']]
<class 'pyspark.rdd.PipelinedRDD'>


In [2]:
numTransacoes = arquivoRDD.count();
print(numTransacoes); #numeros de transações
elementos = arquivoRDD.take(10)
print(elementos)
print(type(elementos));


19
[['1,101,102,103'], ['2,101,102,103,104,105'], ['3,101,103,104'], ['4,101,102,104,105'], ['5,102,103,104'], ['6,103,105,106,107,108'], ['7,103,105,106,107,110'], ['8,104,106,107,109,110'], ['9,106,108,110'], ['10,102,103,107,108,109']]
<class 'list'>


In [3]:
tuplesRDD = arquivoRDD.map(lambda x : (x[0].split(",")[0],  x[0].split(",")[1:]));
#print("Ids:");
#print(tuplesRDD.keys().collect());
#print("Transações:");
#print(tuplesRDD.values().collect());


In [4]:
vertical = tuplesRDD.flatMapValues(lambda x : x) 
verticalTuplesRDD = vertical.map(lambda x : (x[1], x[0])) #inverte disciplina e aluno
aggregateByKeyRDD = verticalTuplesRDD.groupByKey() \
                                     .map(lambda x : (x[0], list(x[1]))).sortByKey();

print(aggregateByKeyRDD.map(lambda x : (x[0], len(x[1]))).take(10));

[('101', 5), ('102', 10), ('103', 11), ('104', 11), ('105', 5), ('106', 5), ('107', 11), ('108', 7), ('109', 11), ('110', 6)]


In [5]:
filterValuesRDD = aggregateByKeyRDD.filter(lambda x : len(x[1]) > minSupport) \
                                   .map(lambda x : ([x[0]],x[1]));

print(filterValuesRDD.map(lambda x : (x[0], len(x[1]))).take(10)); # (código da disciplina, [códigos dos alunos])
#print(filterValuesRDD.count()); #count de disciplinas com mais do que o suporte mínimo



[(['101'], 5), (['102'], 10), (['103'], 11), (['104'], 11), (['105'], 5), (['106'], 5), (['107'], 11), (['108'], 7), (['109'], 11), (['110'], 6)]


In [6]:
def distinct(a, b): return list(set(a) ^ set(b))
def intersect(a, b): return tuple(set(a) & set(b))
def unir(a,b) : return tuple(set(a + b))

In [7]:
def funcaox(disc, rdd_collect):
    list = [];
    for x in rdd_collect:
        if not disc[0] == x[0]:
            disciplinas = unir(disc[0],x[0])
            ids = intersect(disc[1],x[1])
            tupla = [sorted(disciplinas),sorted(ids)]
            if not tupla in list: list += [tupla]
            #tupla = tuple(sorted(disciplinas)),(sorted(ids))
            #if not tupla in list: list += [tupla]
    return list

def funcao(disc, rdd_collect):
    lista = [];
    for x in rdd_collect:
        if (not disc[0] == x[0]) & (disc[0] < x[0]):
            disciplinas = unir(disc[0],x[0])
            ids = intersect(disc[1],x[1])
            tupla = (sorted(disciplinas),sorted(ids))
            if not tupla in lista: lista += [tupla]
            #tupla = tuple(sorted(disciplinas)),(sorted(ids))
            #if not tupla in list: list += [tupla]
    return tuple(lista)
 

In [ ]:
def funcao2x(disc, rdd_collect):
    list = [];
    for x in rdd_collect:
        if not x[0] in disc[0]:
            disciplinas = unir(disc[0],x[0])
            ids = intersect(disc[1],x[1])
            if len(disciplinas) > len(disc[0]):
                tupla = [sorted(disciplinas),sorted(ids)]
                if not tupla in list: list += [tupla]
    return list

def funcao2(disc, rdd_collect):
    lista = [];
    for x in rdd_collect:
        if disc[0] != x[0]:
            disciplinas = unir(disc[0],x[0])
            ids = intersect(disc[1],x[1])
            if (len(disciplinas)) == (len(disc[0]) + 1):
                tupla = (sorted(disciplinas), sorted(ids))
                if not tupla in lista: lista += [tupla]
    return tuple(lista)

In [ ]:
disciplinas_singleton = filterValuesRDD.collect()

In [ ]:
disciplinasRDD = filterValuesRDD.flatMap(lambda x : funcao(x, disciplinas_singleton))

filterDisciplinasRDD = disciplinasRDD.filter(lambda x : len(x[1]) > minSupport) \
                                     .sortByKey()

contagem = filterDisciplinasRDD.map(lambda x : (x[0], len(x[1])))
print(contagem.collect())

In [ ]:
disciplinas2itemsets = filterDisciplinasRDD.collect()

In [ ]:
#disciplinasRDD2 = filterDisciplinasRDD.flatMap(lambda x : funcao2(x, disciplinas_singleton))
disciplinasRDD2 = filterDisciplinasRDD.flatMap(lambda x : funcao2(x, disciplinas2itemsets))

filterDisciplinasRDD2 = disciplinasRDD2.filter(lambda x : len(x[1]) > minSupport) \
                                       .sortByKey()

contagem2 = filterDisciplinasRDD2.map(lambda x : (x[0], len(x[1])))
print(contagem2.collect())

In [ ]:
end = time.time()
print(end - start)